# Data AIS

In [1]:
#For 3.3.1
#Register Sedona Functions to Spark
from sedona.register import SedonaRegistrator
SedonaRegistrator.registerAll(spark)

True

In [2]:
#For 3.3.2
from shapely.geometry import Point, Polygon, mapping
import h3.api.numpy_int as h3int

In [3]:
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql.functions import col, when

In [4]:
# Path
base_path = "s3a://ungp-ais-data-historical-backup/user_temp/"
path_unique = base_path + "222011349/"

In [5]:
# Read Data
data_ais = spark.read.parquet(path_unique + "data-ais-1persen-dunia-2022.parquet")

In [7]:
data_ais.count()

88996473

# Filter Data

In [6]:
# Filter MMSI
filtered_mmsi = data_ais.filter(col("mmsi").between(100000000, 999999999))

In [9]:
filtered_mmsi.count()

88985117

In [7]:
# Filter IMO
filtered_imo = filtered_mmsi.filter(col("imo").between(1000000, 9999999))

In [11]:
filtered_imo.count()

48071225

In [8]:
# Filter Status Navigasi
filtered_nav_status_code = filtered_imo.filter(col("nav_status_code").between(0, 14))

# Nilai yang akan di-filter
nav_status_values = ['Under Way Using Engine', 'At Anchor', 'Restricted Manoeuvrability', 'Moored', 'Engaged In Fishing', 'Underway Sailing']

# Filter data berdasarkan nilai 'nav_status'
filtered_nav_status = filtered_nav_status_code.filter(col("nav_status").isin(nav_status_values))

In [13]:
filtered_nav_status.count()

45613471

In [9]:
# Filter Tipe Kapal
filtered_vessel_type_code = filtered_nav_status.filter(col("vessel_type_code").between(1, 255))

# Nilai yang akan di-filter
vessel_type_values = ['Sailing', 'Tanker', 'Other', 'Pleasure Craft', 'Passenger', 'Fishing', 'Port Tender', 'Dredging', 'Cargo']

# Filter data berdasarkan nilai 'vessel_type'
filtered_vessel_type = filtered_vessel_type_code.filter(col("vessel_type").isin(vessel_type_values))

In [15]:
filtered_vessel_type.count()

38695685

In [10]:
# Filter Negara Kapal
filtered_flag_country_code = filtered_vessel_type.filter(col("flag_code").between(201, 775))

# Filter data berdasarkan nilai 'flag_country'
# bagi jadi dua, Indonesia dan negara asing
filtered_flag_country = filtered_flag_country_code.withColumn("status_country", when(col("flag_country") == "Indonesia", "Indonesia").otherwise("Asing"))

In [17]:
filtered_flag_country.count()

38100151

In [11]:
# Filter Latitude Longitude (tidak perlu)
filtered_lat = filtered_flag_country.filter(col("latitude").between(-90, 90))
filtered_lat_long = filtered_lat.filter(col("longitude").between(-180, 180))

In [19]:
filtered_lat_long.count()

38100151

In [12]:
# Filter dt_pos_utc (tidak perlu) 
pattern = "^[1-9][0-9]{0,3}-[1-9]|0[1-9]|1[0-2]-[1-9]|0[1-9]|[1-2][0-9]|3[0-1] [0-9]|0[0-9]|1[0-9]|2[0-3]:[0-5][0-9]:[0-5][0-9]$"

# Filter data untuk mendapatkan hanya nilai 'dt_pos_utc' yang tidak valid
filtered_dt_pos_utc = filtered_lat_long.filter(F.col("dt_pos_utc").rlike(pattern))

In [21]:
filtered_dt_pos_utc.count()

38100151

In [13]:
# Filter Record Duplikat

# Menghapus record duplikat berdasarkan semua kolom (variabel)
filtered_duplicates = filtered_dt_pos_utc.dropDuplicates()

In [23]:
filtered_duplicates.count()

38099598

In [14]:
# Filter Pergerakan Anomali

# Tentukan kriteria pergerakan tidak normal
kriteria_pergerakan_tidak_normal = (
    ((col("nav_status").isin(['At Anchor', 'Moored'])) &
    (col("sog") > 1)) | 
    ((col("nav_status").isin(['Not Under Command', 'Underway Sailing', 'Under Way Using Engine', 'Engaged In Fishing', 'Restricted Manoeuvrability', 'Aground'])) &
    (col("sog") < 1))
)

# kriteria pergerakan normal
kriteria_pergerakan_normal = ~kriteria_pergerakan_tidak_normal

# Filter pergerakan normal
filtered_pergerakan_normal = filtered_duplicates.filter(kriteria_pergerakan_normal)

In [25]:
filtered_pergerakan_normal.count()

32675659

## Save Data

In [15]:
data_ais_filtered = filtered_pergerakan_normal

In [16]:
# Save Data
data_ais_filtered.write.option("header", True).mode("overwrite").parquet(path_unique + "data-ais-1persen-filter-dunia-2022.parquet")

In [28]:
spark.stop()